In [21]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# dataset_red=pd.read_csv("Data/winequality-red.csv",sep=";",index_col=False)
# dataset_white=pd.read_csv("Data/winequality-white.csv",sep=";",index_col=False)
dataset_red=pd.read_csv("Data/winequality-red.csv",sep=";",index_col=False)
dataset_white=pd.read_csv("Data/winequality-white.csv",sep=";",index_col=False)

red=["red"]*len(dataset_red)
dataset_red.insert(0,"Label",red)
white=["white"]*len(dataset_white)
dataset_white.insert(0,"Label",white)
# dataset_white = dataset_white[:1599]
# dataset_white.head(n=5)
dataset=pd.concat([dataset_red,dataset_white],axis=0)
dataset.set_index("Label")

X=dataset.drop("Label",axis=1)
dataset['Label'].value_counts()

white    4898
red      1599
Name: Label, dtype: int64

In [22]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
Xs=scaler.fit_transform(X)


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
dataset["Label"]=le.fit_transform(dataset["Label"])
y=dataset["Label"]


from sklearn.feature_selection import SelectKBest,f_classif
Xs=SelectKBest(f_classif,k=4).fit_transform(Xs,y)


from sklearn.model_selection import train_test_split, cross_val_score

Xs_train,Xs_test,y_train,y_test=train_test_split(Xs,y,test_size=0.3,random_state=1,stratify=y)



In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import KFold, GridSearchCV
# from sklearn.tree import DecisionTreeClassifier

# clf_fs_cv=DecisionTreeClassifier(criterion="entropy")



# p_grid={"splitter":["best","random"],"max_features":[1,2,3,4],"max_depth":[2,3,4,5,6,7,8]}


# inner_cv=KFold(n_splits=3,shuffle=True)
# outer_cv=KFold(n_splits=5,shuffle=True)


# clf = GridSearchCV(estimator=clf_fs_cv, param_grid=p_grid, cv=inner_cv)
# nested_score = cross_val_score(clf, X=Xs, y=y, cv=outer_cv)

# print(nested_score.mean())
# print(y)

# clf_fs_cv.fit(Xs_train,y_train)
# confusion_matrix(clf_fs_cv.predict(Xs_test),y_test)

In [23]:
from tensorflow import keras

#NN model
model = keras.models.Sequential()
model.add(keras.layers.Dense(256, input_dim=4, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.summary()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(np.array(Xs_train), np.array(y_train), epochs=100, validation_data=(np.array(Xs_test), np.array(y_test)))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 256)               1280      
_________________________________________________________________
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_17 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 257       
Total params: 198,913
Trainable params: 198,913
Non-trainable params: 0
_________________________________________________________________
Train on 4547 samples, validate on 1950 samples
Epoch 1/100
4547/4547 [==============================] - 1s 157us/

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(Xs_test)
print(y_pred)
print(y_test)
y_test = np.array(y_test)
y_test = y_test.reshape(y_test.shape[0], 1)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [0]]
3199    1
3547    1
4503    1
510     1
3413    1
       ..
1054    0
24      1
567     1
3575    1
618     0
Name: Label, Length: 1950, dtype: int32
[[ 464   16]
 [   8 1462]]


0.9876923076923076